In [1]:
import os 

In [2]:
os.chdir("..")

In [3]:
%pwd

'/workspaces/breast_cancer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    train_data_path : Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.xgb
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            train_data_path = config.train_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [22]:
import os
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_score
from mlProject.utils.common import save_json
from urllib.parse import urlparse
import numpy as np
import joblib
from mlProject.entity.config_entity import ModelEvaluationConfig
from pathlib import Path



class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        cm = confusion_matrix(actual, pred)
        accuracy = accuracy_score(actual, pred)
       
        return cm , accuracy
    def k_fold_cross_validation(self, estimator, X, Y, cv=10):
        kfc = cross_val_score(estimator=estimator, X = X, Y = Y, cv=cv)

        return kfc
        
    def save_results(self):
        train_data =pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        
        x_train = train_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (cm , accuracy) = self.eval_metrics(test_y, predicted_qualities)
        
        kfc = self.k_fold_cross_validation(model, x_train,y_train)
        
        mean_accuracy = kfc.mean() * 100
        std_deviation = kfc.std() * 100
        # Saving metrics as local
        scores = {"confusion_matrix": cm,
                  "accuracy_score": accuracy,
                  "mean accuracy": mean_accuracy,
                  "standard deviation": std_deviation }
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [23]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-04-06 20:23:47,666: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-06 20:23:47,667: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-06 20:23:47,669: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-06 20:23:47,670: INFO: common: created directory at: artifacts]
[2024-04-06 20:23:47,671: INFO: common: created directory at: artifacts/model_evaluation]


TypeError: got an unexpected keyword argument 'Y'